In [1]:
# Imports
from Code.h5Constructor import h5Constructor

In [2]:
gc = h5Constructor('./Dataset/CIFs/Train/', './Dataset/h5/Train_GPU/')

In [3]:
gc.gen_h5s(parallelize=False)


Constructing graphs from cif files:


  0%|          | 0/28 [00:00<?, ?it/s]















  4%|▎         | 1/28 [00:34<15:29, 34.42s/it]
















  7%|▋         | 2/28 [01:30<20:21, 46.96s/it]











 11%|█         | 3/28 [02:03<16:58, 40.73s/it]











 14%|█▍        | 4/28 [02:30<14:11, 35.48s/it]











 18%|█▊        | 5/28 [02:59<12:37, 32.91s/it]











 21%|██▏       | 6/28 [03:25<11:12, 30.55s/it]









 25%|██▌       | 7/28 [03:42<09:12, 26.29s/it]
















 29%|██▊       | 8/28 [04:18<09:48, 29.44s/it]


















In [1]:
from Code.datasetClass import InOrgMatDatasets

test = InOrgMatDatasets('DatasetTest', root='./Dataset/', force_update=False)

Extracting Dataset/DatasetTest/raw/DatasetTest.zip
Processing...
Done!


In [2]:
test.get(15, data_split='train')

Data(
  x=[12, 4],
  edge_index=[2, 64],
  edge_attr=[88],
  y={
    crystal_type='Flourite',
    atomic_species=[2],
    cell_params=[6],
    np_size=11.59610962646957,
    nd=[2, 580],
    xrd=[2, 580],
    nPDF=[2, 301],
    xPDF=[2, 301],
    sans=[2, 300],
    saxs=[2, 300]
  },
  pos=[12, 3],
  pos_real=[12, 3]
)